<a href="https://colab.research.google.com/github/nina-prog/DataAnalysis_VAE/blob/main/VAE_v2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

To-Do


*   Layer von z_mean und z_log_var entsprechende Aktivierungsfunktionen
*   nach encoder.summary (), encoder.compile(optimizer= …., loss=…) zu machen. (Das selber mit dem Decoder). Dann kann man nach trainieren des Modells : encoder.predict(DATAFRAME) machen und enthält damit die z-Werte für die Input Daten. Sonst gibt es in Tensorflow noch die Möglichkeit Layer direkt zu tracken
*    plottet man nämlich den Trainings Loss gegen den Validation Loss. Ihr habt keinen validation loss (hab gerade in den code geschaut), dann könnt ihr vae.evaluate(x_test) machen und den loss von .fit gegen den loss von .evaluate plotten. (Das gleiche könnt ihr dann natürlich auch für den reconstruction loss und die kl_divergence machen, müsst ihr aber nicht. Fürs erste ist der vollständige loss.)



# Data Preprocessing

---



## Load Data

In [ ]:
### Load ecg5000 data using read_csv
ecg5000 = pd.read_csv('ECG5000_ALL.txt', sep='\s+', header=None)

### Delete label-column first (column 0)
ecg5000.drop(ecg5000.columns[[0]], axis=1, inplace=True)

In [ ]:
### Optional test and info about data set
print("Type of ecg5000: \t \t {}".format(type(ecg5000)))
print("Dimensions of ecg5000: \t \t {}".format(ecg5000.shape))
print("Number of elements of ecg5000: \t {}".format((ecg5000.size)))
print("Display first 10 rows of ecg5000: \n {}".format(ecg5000.head(10)))

## Scale Data

In [ ]:
### Normalize dataframe with min-max-normalization to range between [-0.8, 0.8] using sklearn MinMaxScaler
min_max_scaler = MinMaxScaler(feature_range=(-0.8,0.8))
scaled_ecg5000 = pd.DataFrame(min_max_scaler.fit_transform(ecg5000))
print(scaled_ecg5000)

## Split Data

In [ ]:
### Split Data into 80/20 Training, Test
trainDF, testDF = train_test_split(scaled_ecg5000, test_size=0.2)
# Optional test and info about new data sets
print("Shape of Training DataFrame: \t {}".format(trainDF.shape))
print("Shape of Test DataFrame: \t {}".format(testDF.shape))

## Reshape Data

In [ ]:
### Convert to array
x_train = trainDF.to_numpy()
x_test = testDF.to_numpy()

### Reshape input into [samples, timesteps, features]
s_train = len(trainDF.index) # samples
s_test = len(testDF.index) # samples
n_train = len(trainDF.columns) # time steps
n_test = len(testDF.columns) # time steps
x_train = x_train.reshape(s_train, n_train, 1)
x_test = x_test.reshape(s_test, n_test, 1)

### Properties
print("Shape of reshaped train dataset: {}".format(x_train.shape))
print("Shape of reshaped test dataset: {}".format(x_test.shape))

# Create Sample Layer

---



In [ ]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z"""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Build Variational Autoencoder (VAE)

---



## Define Model

In [ ]:
### For better understanding visit: https://towardsdatascience.com/autoencoders-for-the-compression-of-stock-market-data-28e8c1a2da3e
### For better understanding of layers and Recreating auto encoders visit: https://machinelearningmastery.com/lstm-autoencoders/
### or for code: https://gist.github.com/GerardBCN/40349b39bc45d4550141aff6966d1619#file-stock_price_autoencoding-ipynb
### For Reshaping Issues: https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/

### fit model
encoding_dim = 140
latent_dim = 2
epochs = 50
batch_size = 32

## Encoder

In [ ]:
### Define Encoder Layers
encoder_inputs = keras.Input(shape=(140, 1), name='Encoder_Input_layer')

encoded = Bidirectional(layers.LSTM(encoding_dim, activation='tanh', name=''), name='Encode_1')(encoder_inputs)
encoded = layers.Dense(5, activation='tanh', name='Encode_2')(encoded) #5, because 5 class in data ecg5000 - evtl 2,1
z_mean = layers.Dense(latent_dim, activation="sigmoid" name="z_mean")(encoded) 
z_log_var = layers.Dense(latent_dim, activation="sigmoid" name="z_log_var")(encoded)

z = Sampling(name='Sample_layer')([z_mean, z_log_var])

### Build Encoder
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

encoder.summary()
# configure encoder for training
encoder.compile(optimizer='adam', loss='binary_crossentropy')

plot_model(encoder, show_shapes=True, to_file='reconstruct_lstm_encoder.png')

## Decoder

In [ ]:
### Define Decoder Layers
latent_inputs = keras.Input(shape=(latent_dim,), name='Decoder_Input_layer')

decoded = layers.Dense(140, activation='tanh', name='Decode_1')(latent_inputs)
decoded = layers.Reshape((140,1), name='Decode_2')(decoded)
decoded = Bidirectional(layers.LSTM(encoding_dim, return_sequences=True, activation='tanh', name=''), name='Decode_3')(decoded)

decoder_outputs = TimeDistributed(Dense(1, activation='tanh', name=''),name='Decoder_Output_Layer')(decoded)

### Build Decoder
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

decoder.summary()
# configure decoder for training
decoder.compile(optimizer='adam', loss='binary_crossentropy')

plot_model(decoder, show_shapes=True, to_file='reconstruct_lstm_decoder.png')

## Connecting the Encoder and Decoder

In [ ]:
### For better understanding visit https://becominghuman.ai/using-variational-autoencoder-vae-to-generate-new-images-14328877e88d

### Instantiate VAE model
# The output of vae model is the output of decoder in which its input is taken from the output of encoder !
decoder_outputs = decoder(encoder(encoder_inputs))
vae = keras.Model(encoder_inputs, decoder_outputs, name='vae_v2_0')

vae.summary()
plot_model(vae, show_shapes=True, to_file='reconstruct_lstm_variational_autoencoder.png')

# Train VAE

---



## Define Train Model

In [ ]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = encoder(data)
            reconstruction = decoder(z)
            # reconstruction_loss = distance between Input and Output
            reconstruction_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(data, reconstruction)
            )
            reconstruction_loss *= 28 * 28
            # kl_loss = distance between distributions and thus ensures the regular laten space
            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *= -0.5
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

## Train

In [ ]:
### To improve training see here: https://becominghuman.ai/using-variational-autoencoder-vae-to-generate-new-images-14328877e88d
 
### Train
vae = VAE(encoder, decoder, name="VAE")
vae.compile(optimizer='adam', loss='binary_crossentropy')
history = vae.fit(x_train, epochs=epochs, batch_size=batch_size)

### Recreation
bottleneck = encoder.predict(x_train) # z values
decoded_ecg5000 = decoder.predict(bottleneck)

In [ ]:
### Display the training progress
# Loss vs Reconstruction_loss
plt.figure(figsize=(20,12))
plt.plot(history.history['loss'])
plt.plot(history.history['reconstruction_loss'])
plt.legend(["Loss", "Reconstruction Loss"])
plt.xlabel("Epoch")
plt.title("Loss vs. Reconstruction Loss")


In [ ]:
# Train loss vs val loss
# Returns the loss value & metrics values for the model in test mode
val_loss = vae.evaluate(x_test)

plt.figure(figsize=(20,12))
plt.plot(history.history['loss'])
plt.plot(val_loss['val_loss'])
plt.legend(["Loss", "Validation Loss"])
plt.xlabel("Epoch")
plt.title("Loss vs. Validation Loss")

In [ ]:
##################################################################################### TODO: Display latent space
### Encode data into latent space and show the distribution using simple scatter plot
bottleneck = encoder.predict(x_train)

print(bottleneck)
print(type(bottleneck))
print(len(bottleneck))

array0 = bottleneck[0]
print(len(array0))
print(array0.shape)
array1 = bottleneck[1]
print(len(array1))
print(array1.shape)
array2 = bottleneck[2]
print(len(array2))
print(array2.shape)

In [ ]:
# *i unpacks i into a tuple (i[0], i[1]), which is interpreted as (x,y) by plt.scatter
# for a in bottleneck:
#  for i in a:
#   plt.scatter(*i)

#x = array1[:,0]
#y = array1[:,1]

plt.figure(figsize=(18,12))
plt.scatter(array0[:,0], array0[:,1], s=2)
plt.grid()
plt.show()

# Plot Results

---



In [ ]:
### Test if Input fits Dim of Output
print("Shape of Input x_train: {}".format(x_train.shape))
print("Shape of Output y_train: {}".format(decoded_ecg5000.shape))

### Covert to 2D Array -- ("-1" = make a dimension (here rows) the size that will use the remaining unspecified elements)
new_x_train= x_train.reshape(-1,140)
new_decoded_ecg5000 = decoded_ecg5000.reshape(-1,140)

print("Shape of Input after reshaping: {}".format(new_x_train.shape))
print("Shape of Output after reshaping: {}".format(new_decoded_ecg5000.shape))

In [ ]:
### Plot only one sample
i = 34 # indize/sample which is going to be plotted
plt.figure(linewidth = 1, figsize=(25,6))
plt.title('Autoencoder Result')
plt.xlabel('time steps')
plt.plot(new_x_train[i], label='original ecg5000')
plt.plot(new_decoded_ecg5000[i], label='decoded ecg5000')
plt.legend(loc="upper left")
plt.show()

In [ ]:
### Plot Multiple Samples
n_rows = 2                   
n_cols = 2

# Size Properties and layout design for tighter representation
fig, axs = plt.subplots(nrows = n_rows, ncols = n_cols, figsize=(23,20))
fig.tight_layout(w_pad=4, h_pad = 5)

# Subplotting
i = 5
for row in range(n_rows):
  for col in range(n_cols):
    axs[row, col].plot(new_x_train[i])
    axs[row, col].plot(new_decoded_ecg5000[i])
    axs[row, col].legend(["Original ECG5000 Sample {}".format(i), "Decoded ECG5000 Sample {}".format(i)])
    axs[row, col].set(xlabel = "Time Steps", ylabel = "Heartbeat Interpolated", title = "Sample {}".format(i))
    i = i + 1
